In [ ]:
pip install meteostat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from datetime import datetime
from meteostat import Stations
from meteostat import Hourly

In [ ]:
from google.colab import files
uploaded = files.upload()
import io

Saving UFO_cleaned.csv to UFO_cleaned (1).csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['UFO_cleaned.csv']))

In [ ]:
df.head()

,city,state,date_time,shape,text,city_latitude,city_longitude,year,month,day,hour
0,Chester,VA,2019-12-12 18:43:00,light,My wife was driving southeast on a fairly popu...,37.343152,-77.408582,2019,12,12,18
1,Rocky Hill,CT,2019-03-22 18:30:00,circle,I think that I may caught a UFO on the NBC Nig...,41.664800,-72.639300,2019,3,22,18
2,Ottawa,ON,2019-04-17 02:00:00,teardrop,I was driving towards the intersection of fall...,45.381383,-75.708501,2019,4,17,2
3,Kirbyville,TX,2019-04-02 20:25:00,disk,"The object has flashing lights that are green,...",30.677200,-94.005200,2019,4,2,20
4,Tucson,AZ,2019-05-01 11:00:00,unknown,"Description is the same as Washington, DC, eve...",32.259941,-110.927542,2019,5,1,11


In [ ]:
# Add the new columns regarding weather and datetime breakdown 
header_list = ['city','state', 'date_time', 'shape', 'text', 'posted', 'city_latitude', 'city_longitude', 'year', 'month', 'day', 'hour', 'temperature', 'relative_humidity', 'precipitation', 'snow', 'wind_direction', 'wind_speed']
df = df.reindex(columns = header_list)  

In [ ]:
from dateutil.relativedelta import relativedelta
import operator
from tqdm import tqdm

In [ ]:
num_rows = len(df)
for i in tqdm(range(num_rows)):
  # adds the weather variables for each row to the dataframe
  my_lat = df.at[i, 'city_latitude']
  my_long = df.at[i, 'city_longitude']
  stations = Stations()
  stations = stations.nearby(my_lat, my_long)
  station = stations.fetch(1) # get closest weather station 
  closest_weather_station_id = station.index.values[0] # get the id of closest weather station

  year = df.at[i, 'year'] 
  month = df.at[i, 'month']
  day = df.at[i, 'day']
  hour = df.at[i, 'hour']

  start = end = datetime(year, month, day, hour) # we just want one hour of weather data 
  weather_data = Hourly(closest_weather_station_id, start, end) 
  weather_data = weather_data.fetch()
  if not weather_data.empty: # some weather stations just won't have data on the specific hour and location we have 
    weather_data = weather_data.loc[:, ['temp', 'rhum', 'prcp', 'snow', 'wdir', 'wspd']] # get these columns for that hour 

    my_temp = weather_data['temp'].loc[weather_data.index[0]]
    my_rhum = weather_data['rhum'].loc[weather_data.index[0]]
    my_prcp = weather_data['prcp'].loc[weather_data.index[0]]
    my_snow = weather_data['snow'].loc[weather_data.index[0]]
    my_wdir = weather_data['wdir'].loc[weather_data.index[0]]
    my_wspd = weather_data['wspd'].loc[weather_data.index[0]]

    df.at[i, 'temperature'] = my_temp
    df.at[i, 'relative_humidity'] = my_rhum
    df.at[i, 'precipitation'] = my_prcp
    df.at[i, 'snow'] = my_snow
    df.at[i, 'wind_direction'] = my_wdir
    df.at[i, 'wind_speed'] = my_wspd

print(df.to_string())

100%|██████████| 22482/22482 [41:57<00:00,  8.93it/s]
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
df.head(20)

,city,state,date_time,shape,text,posted,city_latitude,city_longitude,year,month,day,hour,temperature,relative_humidity,precipitation,snow,wind_direction,wind_speed
0,Chester,VA,2019-12-12 18:43:00,light,My wife was driving southeast on a fairly popu...,NaN,37.343152,-77.408582,2019,12,12,18,3.9,46.0,0.0,NaN,NaN,9.4
1,Rocky Hill,CT,2019-03-22 18:30:00,circle,I think that I may caught a UFO on the NBC Nig...,NaN,41.664800,-72.639300,2019,3,22,18,5.6,82.0,0.8,NaN,310.0,9.4
2,Ottawa,ON,2019-04-17 02:00:00,teardrop,I was driving towards the intersection of fall...,NaN,45.381383,-75.708501,2019,4,17,2,NaN,NaN,NaN,NaN,NaN,NaN
3,Kirbyville,TX,2019-04-02 20:25:00,disk,"The object has flashing lights that are green,...",NaN,30.677200,-94.005200,2019,4,2,20,20.0,26.0,0.0,NaN,350.0,7.6
4,Tucson,AZ,2019-05-01 11:00:00,unknown,"Description is the same as Washington, DC, eve...",NaN,32.259941,-110.927542,2019,5,1,11,13.9,55.0,0.0,NaN,180.0,0.0
5,Gold Canyon,AZ,2019-04-10 17:00:00,circle,Apr. 10th we witnessed a very bright silvery r...,NaN,33.371500,-111.436900,2019,4,10,17,19.0,21.0,NaN,NaN,260.0,27.7
6,Brookville,IN,2019-06-18 21:00:00,sphere,Metal orb of wires that was seen through a tel...,NaN,39.404700,-84.973600,2019,6,18,21,NaN,NaN,NaN,NaN,NaN,NaN
7,Melbourne Beach,FL,2019-06-12 22:00:00,unknown,We think 2 UFOs....2 tiny lights recorded for ...,NaN,27.966800,-80.509600,2019,6,12,22,NaN,NaN,NaN,NaN,NaN,NaN
8,Carrizozo,NM,2019-06-11 22:00:00,changing,I was driving and saw three glowing orbs is th...,NaN,33.784900,-105.833200,2019,6,11,22,26.0,23.0,NaN,NaN,350.0,16.6
9,Waco,TX,2018-06-15 01:00:00,circle,I was in pool and my wife was sitting on the e...,NaN,31.573598,-97.135849,2018,6,15,1,33.9,38.0,0.0,NaN,150.0,25.9


In [ ]:
len(df)

22482

In [ ]:
df.isna().sum()

city                     0
state                    0
date_time                0
shape                 1040
text                    15
city_latitude            0
city_longitude           0
year                     0
month                    0
day                      0
hour                     0
temperature           5466
relative_humidity     5486
precipitation        11342
snow                 22482
wind_direction        8631
wind_speed            5643
dtype: int64

In [ ]:
df.drop('posted', axis=1, inplace=True)

In [ ]:
df.to_csv('UFO_and_Weather.csv')